In [ ]:
#default_exp utils.misc

In [ ]:
#export

import os, sys
import inspect
import logging
import json

import tensorflow as tf

from typing import Any, Union

# Debug setup

In [ ]:
#DEBUG = os.environ.get('GITHUB_ACTIONS', False) or os.environ.get('AUDIOENGINE_DEBUG', False)
DEBUG = True

# Object attribute util functions

In [ ]:
#export

def recursive_get_size(obj: Any, seen: Union[None, set] = None) -> int:
    '''
    Get the size of an object in memory recursivly
    
    Inputs:
        param: obj |Any| (Any python object)
        kwarg: seen |set| (A set of ids of python objects already seen by the function. This should not be done manually)
    Outputs:
        return: size |int| (The size of the object in memory measured in bytes)
    '''
    size = sys.getsizeof(obj)
    seen = (set() if seen is None else seen)
    if(id(obj) in seen): return 0 #In the case of a self reference
    else: pass
    seen.add(id(obj))
    if(isinstance(obj, dict)):
        size += sum([recursive_get_size(v, seen=seen) for v in obj.values()])
        size += sum([recursive_get_size(k ,seen=seen) for k in obj.keys()])
    elif(hasattr(obj, '__dict__')):
        size += recursive_get_size(obj.__dict__, seen)
    elif(hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray))):
        size += sum([recursive_get_size(i, seen) for i in obj])
    return size

# Logging utils

In [ ]:
#export

def log_init(log_filepath: str = '', log_level: str = '', clear_existing_log: bool = False):
    '''
    Init the logging module and set up where to log to. When no kwargs are passed the function will check the 
    environment variable AUDIOENGINE_CONF_JSON for a path to a file to log to. If it doesn't exist then the default
    will be grabbed from /project/Development/ML/audio/config/audioengine.conf. If it isn't in there then it will
    log to console. It also sets what level is written to the logs. The options are 
    [debug, info, warning, error, critical] this argument is case insensitive.
    
    Inputs:
        kwarg: log_filepath |str| (Optional filepath to log to)
        kwarg: log_level |str| (Optional level of log sensitivity)
        kwarg: clear_existing_log |bool| (Optional overwrite existing file or just append. True will overwrite.) 
    Outputs:
        If the filepath to the log file doesn't exist it will be created and opened
    '''
    config_filepath = os.environ.get('AUDIOENGINE_CONF_JSON', '/project/Development/ML/audio/config/audioengine.conf')
    if(os.path.isfile(config_filepath)):
        config_fp = open(config_filepath, 'r')
        config_str = config_fp.read()
        config_fp.close()
    else:
        config_str = ''
    config_json = json.loads(config_str)
    logging_options_json = config_json.get('logging', None)
    if(logging_options_json is not None and (log_filepath == '' or log_level == '')):
        if(log_level == ''):
            log_level = logging_options_json.get('log_level')
        else:
            pass
        if(log_filepath == ''):
            log_filepath = logging_options_json.get('log_file')
        else:
            pass
    else:
        if(log_level == ''):
            log_level = 'warning'
        else:
            pass
        #Don't do anything for the filepath because it needs to log to console
        
    filemode = ('w' if clear_existing_log else 'a')
    
    if(log_level.lower() == 'debug'):
        if(log_filepath == ''):
            logging.basicConfig(level=logging.DEBUG, filemode=filemode)
        else:
            logging.basicConfig(level=logging.DEBUG, filename=log_filepath, filemode=filemode)
    elif(log_level.lower() == 'info'):
        if(log_filepath == ''):
            logging.basicConfig(level=logging.INFO, filemode=filemode)
        else:
            logging.basicConfig(level=logging.INFO, filename=log_filepath, filemode=filemode)
    elif(log_level.lower() == 'warning'):
        if(log_filepath == ''):
            logging.basicConfig(level=logging.WARNING, filemode=filemode)
        else:
            logging.basicConfig(level=logging.WARNING, filename=log_filepath, filemode=filemode)
    elif(log_level.lower() == 'error'):
        if(log_filepath == ''):
            logging.basicConfig(level=logging.ERROR, filemode=filemode)
        else:
            logging.basicConfig(level=logging.ERROR, filename=log_filepath, filemode=filemode)
    elif(log_level.lower() == 'critical'):
        if(log_filepath == ''):
            logging.basicConfig(level=logging.CRITICAL, filemode=filemode)
        else:
            logging.basicConfig(level=logging.CRITICAL, filename=log_filepath, filemode=filemode)
    else:
        #The log_level wasn't matched
        logging.basicConfig(level=logging.WARNING)
    log_location = ('console' if log_filepath == '' else 'file: {}'.format(log_filepath))
    print('[INFO] Logging now set to {} with level {}'.format(log_location, log_level.upper()))
        

def log_critical(message: str, exc_info: bool = False):
    '''
    Log a critical event to the current log and optionally include a traceback
    
    Inputs:
        param: message |str| (The message to print to the log)
        kwarg: exc_info |bool| (Include the traceback. False won't include the traceback and True will.)
    Outputs:
        Prints message and a little bit of context info to the log
    '''
    func = inspect.currentframe().f_back.f_code
    # Dump the message + the name of this function to the log.
    logging.critical("{}: {} in {}:{}".format(func.co_filename, func.co_name, func.co_firstlineno, message),
                     exc_info=exc_info)

def log_error(message: str, exc_info: bool = False):
    '''
    Log a error event to the current log and optionally include a traceback
    
    Inputs:
        param: message |str| (The message to print to the log)
        kwarg: exc_info |bool| (Include the traceback. False won't include the traceback and True will.)
    Outputs:
        Prints message and a little bit of context info to the log
    '''
    func = inspect.currentframe().f_back.f_code
    # Dump the message + the name of this function to the log.
    logging.error("{}: {} in {}:{}".format(func.co_filename, func.co_name, func.co_firstlineno, message),
                  exc_info=exc_info)
    
def log_warning(message: str):
    '''
    Log a warning event to the current log and optionally include a traceback
    
    Inputs:
        param: message |str| (The message to print to the log)
    Outputs:
        Prints message and a little bit of context info to the log
    '''
    func = inspect.currentframe().f_back.f_code
    # Dump the message + the name of this function to the log.
    logging.warning("{}: {} in {}:{}".format(func.co_filename, func.co_name, func.co_firstlineno, message))    
    
def log_info(message: str):
    '''
    Log a info event to the current log and optionally include a traceback
    
    Inputs:
        param: message |str| (The message to print to the log)
    Outputs:
        Prints message and a little bit of context info to the log
    '''
    func = inspect.currentframe().f_back.f_code
    # Dump the message + the name of this function to the log.
    logging.info("{}: {} in {}:{}".format(func.co_filename, func.co_name, func.co_firstlineno, message))
    
def log_debug(message: str):
    '''
    Log a debug event to the current log and optionally include a traceback
    
    Inputs:
        param: message |str| (The message to print to the log)
    Outputs:
        Prints message and a little bit of context info to the log
    '''
    func = inspect.currentframe().f_back.f_code
    # Dump the message + the name of this function to the log.
    logging.debug("{}: {} in {}:{}".format(func.co_filename, func.co_name, func.co_firstlineno, message))

In [ ]:
if DEBUG:
    #log_init()
    log_init(clear_existing_log=True)
    log_critical('Critical test')
    log_error('Error test')
    log_warning('Warning test')
    log_info('Info test')
    log_debug('Debug test')
    
    try:
        a = 2 / 0
    except Exception as exception:
        log_critical('Exception raised', exc_info=True)
        
    try:
        a = 2 / 0
    except Exception as exception:
        log_error('Exception raised', exc_info=True)

# File utils

In [ ]:
#export

def get_json_file_integrity(original_json: dict, filepath: str) -> bool:
    '''
    Load the file from disk and check that it matches the original json that was written to the disk to check that
    the file was written without error
    
    Inputs:
        param: original_json |dict| (The json dictionary that was written to disk)
        param: filepath |str| (The filepath to where the json dictionary was written)
    Outputs:
        return: valid |bool| (True if the file matches the json written to it and False otherwise)
    '''
    json_file = open(filepath, 'r')
    json_from_disk = json.loads(json_file.read())
    json_file.close()
    if(original_json == json_from_disk):
        return True
    else:
        return False
    return False

In [ ]:
#export

def change_file_extension(filename: str, extension: str) -> str:
    '''
    Strip the last characters before the last dot and append the new extension to the filename
    
    Inputs:
        param: filename |str| (The current filename with the old extension)
        param: extension |str| (The new extension to be put on the string)
    Outputs:
        return: new_filename |str| (The filename with the new extension)
    '''
    filename_split = filename.split('.')
    filename_split.pop(len(filename_split)-1)
    new_filename = '.'.join(filename_split) + extension
    return new_filename

# Tensor utils

In [ ]:
#export

def pad_up_to(t, max_in_dims, constant_values):
    s = tf.shape(t)
    paddings = [[0, m-s[i]] for (i,m) in enumerate(max_in_dims)]
    return tf.pad(t, paddings, 'CONSTANT', constant_values=constant_values)